<a href="https://colab.research.google.com/github/LilianYou/Sea_Hero_Quest/blob/main/Chi_Square_Test_Cluster_shared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import pandas as pd
import numpy as np
import seaborn as sns
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.datasets import load_iris
import matplotlib.pyplot as plt

In [19]:
# df = pd.read_csv('/content/drive/My Drive/SHQ/Data/cfa_test_df.csv')


df0 = pd.read_csv('/content/drive/My Drive/SHQ/Data/2019_cleaned_demographics_3121970.csv')


In [21]:
df = df0.drop(['navigating_skills'], axis = 1)

In [22]:
df.head()

,age,education,gender,hand,home_environment,sleep,travel_time,nav_num
0,68.0,1,0,0,2,8.0,0,1
1,21.0,1,0,0,1,7.0,1,2
2,36.0,1,1,0,0,5.0,1,3
3,19.0,0,1,0,1,8.0,0,2
4,61.0,1,1,0,1,7.0,0,2


## Step1. Clustering


In [6]:
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.preprocessing import scale,RobustScaler,StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
#import mglearn
import matplotlib.pyplot as plt

In [23]:
# MinMax scaled
scaler = MinMaxScaler(feature_range=(0,1))
scaler.fit(df)
df2 = scaler.transform(df)
df2 = pd.DataFrame(df2,index=df.index,columns=['age',	'gender',	'hand',	'nav_num',	'sleep',	'edu_level',	'commute_level',	'home_env'])
df2[:10]

,age,gender,hand,nav_num,sleep,edu_level,commute_level,home_env
0,0.960784,1.0,0.0,0.0,1.0,0.555556,0.0,0.333333
1,0.039216,1.0,0.0,0.0,0.5,0.444444,0.5,0.666667
2,0.333333,1.0,1.0,0.0,0.0,0.222222,0.5,1.000000
3,0.000000,0.0,1.0,0.0,0.5,0.555556,0.0,0.666667
4,0.823529,1.0,1.0,0.0,0.5,0.444444,0.0,0.666667
5,0.196078,1.0,1.0,0.0,0.0,0.444444,1.0,1.000000
6,0.000000,1.0,0.0,0.0,0.5,0.555556,0.0,0.333333
7,0.372549,1.0,1.0,0.0,0.0,0.444444,0.0,0.666667
8,0.392157,1.0,1.0,0.0,1.0,0.333333,0.0,1.000000
9,0.490196,0.0,1.0,0.0,0.5,0.444444,0.5,0.666667


In [24]:
test = scaler.transform(df)

In [25]:
df2.head()

,age,gender,hand,nav_num,sleep,edu_level,commute_level,home_env
0,0.960784,1.0,0.0,0.0,1.0,0.555556,0.0,0.333333
1,0.039216,1.0,0.0,0.0,0.5,0.444444,0.5,0.666667
2,0.333333,1.0,1.0,0.0,0.0,0.222222,0.5,1.000000
3,0.000000,0.0,1.0,0.0,0.5,0.555556,0.0,0.666667
4,0.823529,1.0,1.0,0.0,0.5,0.444444,0.0,0.666667


In [27]:
df2 = df2.drop(columns="nav_num", axis=1)

In [28]:
df2.head()

,age,gender,hand,sleep,edu_level,commute_level,home_env
0,0.960784,1.0,0.0,1.0,0.555556,0.0,0.333333
1,0.039216,1.0,0.0,0.5,0.444444,0.5,0.666667
2,0.333333,1.0,1.0,0.0,0.222222,0.5,1.000000
3,0.000000,0.0,1.0,0.5,0.555556,0.0,0.666667
4,0.823529,1.0,1.0,0.5,0.444444,0.0,0.666667


In [29]:
km = KMeans(n_clusters= 4, n_init = 2, init = "random", random_state = 42)
km.fit(df2)
y_kmeans = km.predict(df2)
y_kmeans

array([1, 1, 2, ..., 3, 0, 0], dtype=int32)

In [34]:
df_model = df.copy()

skill_map = {0:"very bad", 1:"bad", 2:"good", 3:"very good"}
df_model['nav_num'] = df_model['nav_num'].replace(skill_map)

df_model['cluster'] = y_kmeans

### Step2. Chi-Square of Independence Test

In [35]:
myCrosstable = pd.crosstab(df_model['cluster'], df_model['nav_num'])
myCrosstable

nav_num,bad,good,very bad,very good
cluster,,,,
0,6626,54794,1010,53592
1,44129,153192,8574,59061
2,15457,105254,2198,84819
3,22800,107551,4198,48373


In [36]:
# !pip install scipy

from scipy.stats import chi2_contingency

chiVal, pVal, df, exp = chi2_contingency(myCrosstable)
chiVal, pVal, df, exp

(41069.5070743028,
 0.0,
 9,
 array([[ 13383.84592576,  63270.14235098,   2402.75308828,
          36965.25863499],
        [ 30564.29195415, 144488.14739227,   5487.09595816,
          84416.46469542],
        [ 23962.69282089, 113280.06869631,   4301.93492201,
          66183.30356078],
        [ 21101.1692992 ,  99752.64156044,   3788.21603156,
          58279.9731088 ]]))

In [37]:
colTotals = myCrosstable.sum()
nCols = len(colTotals)

nCols, colTotals

(4, nav_num
 bad           89012
 good         420791
 very bad      15980
 very good    245845
 dtype: int64)

In [38]:
rowTotals = myCrosstable.sum(axis=1)
nRows = len(rowTotals)

nRows, rowTotals

(4, cluster
 0    116022
 1    264956
 2    207728
 3    182922
 dtype: int64)

In [39]:
n = sum(rowTotals)
n

771628

## Step3. Posthoc Analyses with Multiple Comparison

In [40]:
for i in range(nRows):
    for j in range(nCols):
        AdjRes = (myCrosstable.iloc[i,j] - exp[i,j]) / (exp[i,j]*(1-rowTotals[i]/n)*(1-colTotals[j]/n))**0.5
        print(AdjRes)

-67.37768864228899
-54.21664762211999
-31.14915775095124
113.65630039951444
101.80276538994966
41.90706526609468
51.968072523382304
-130.46657165223644
-68.3377354825272
-41.36950815454613
-37.918185487175606
102.65380885045312
14.235323974771413
41.9224874604567
7.702586318613608
-56.91631708396889


In [41]:
phRes = pd.DataFrame(columns=[ 'Group', 'Skills','Adj. Res.'])
for i in range(nRows):
    for j in range(nCols):
        AdjRes = (myCrosstable.iloc[i,j] - exp[i,j]) / (exp[i,j]*(1-rowTotals[i]/n)*(1-colTotals[j]/n))**0.5
        phRes = phRes.append({'Group':myCrosstable.index[i], 'Skills':myCrosstable.columns[j], 'Adj. Res.':AdjRes}, ignore_index=True)

phRes

,Group,Skills,Adj. Res.
0,0,bad,-67.377689
1,0,good,-54.216648
2,0,very bad,-31.149158
3,0,very good,113.656300
4,1,bad,101.802765
5,1,good,41.907065
6,1,very bad,51.968073
7,1,very good,-130.466572
8,2,bad,-68.337735
9,2,good,-41.369508


In [42]:
# !pip install scipy
from scipy.stats import norm

In [43]:
phRes['Sig.'] = 2*(1-norm.cdf(abs(phRes['Adj. Res.'])))
phRes

,Group,Skills,Adj. Res.,Sig.
0,0,bad,-67.377689,0.000000e+00
1,0,good,-54.216648,0.000000e+00
2,0,very bad,-31.149158,0.000000e+00
3,0,very good,113.656300,0.000000e+00
4,1,bad,101.802765,0.000000e+00
5,1,good,41.907065,0.000000e+00
6,1,very bad,51.968073,0.000000e+00
7,1,very good,-130.466572,0.000000e+00
8,2,bad,-68.337735,0.000000e+00
9,2,good,-41.369508,0.000000e+00


In [44]:
phRes['Adj. Sig.'] = phRes.shape[0]*phRes['Sig.']
phRes

,Group,Skills,Adj. Res.,Sig.,Adj. Sig.
0,0,bad,-67.377689,0.000000e+00,0.000000e+00
1,0,good,-54.216648,0.000000e+00,0.000000e+00
2,0,very bad,-31.149158,0.000000e+00,0.000000e+00
3,0,very good,113.656300,0.000000e+00,0.000000e+00
4,1,bad,101.802765,0.000000e+00,0.000000e+00
5,1,good,41.907065,0.000000e+00,0.000000e+00
6,1,very bad,51.968073,0.000000e+00,0.000000e+00
7,1,very good,-130.466572,0.000000e+00,0.000000e+00
8,2,bad,-68.337735,0.000000e+00,0.000000e+00
9,2,good,-41.369508,0.000000e+00,0.000000e+00


In [45]:
phRes.loc[phRes['Adj. Sig.']> 1, 'Adj. Sig.'] = 1
phRes

,Group,Skills,Adj. Res.,Sig.,Adj. Sig.
0,0,bad,-67.377689,0.000000e+00,0.000000e+00
1,0,good,-54.216648,0.000000e+00,0.000000e+00
2,0,very bad,-31.149158,0.000000e+00,0.000000e+00
3,0,very good,113.656300,0.000000e+00,0.000000e+00
4,1,bad,101.802765,0.000000e+00,0.000000e+00
5,1,good,41.907065,0.000000e+00,0.000000e+00
6,1,very bad,51.968073,0.000000e+00,0.000000e+00
7,1,very good,-130.466572,0.000000e+00,0.000000e+00
8,2,bad,-68.337735,0.000000e+00,0.000000e+00
9,2,good,-41.369508,0.000000e+00,0.000000e+00
